# Optimisation d'un réseau de livraison 

## Projet python 1A S2

Travail du binôme Augustin Cablant - Iokanaan Belfis

### Fichier Main

In [ ]:
################  Fichier main.py  ################   
from graph import *
from trucks import *
import sys
sys.setrecursionlimit(10000)


# Route.out 

def recuperation_routes(filename):
    with open(filename) as file: 
        nb_routes = int(file.readline())
        liste_routes = []
        for i in range(int(nb_routes)):
            ligne = file.readline().split()
            liste_routes.append([int(ligne[0]),int(ligne[1]), int(ligne[2])])
        return liste_routes

# Création des fichiers route.out 

for i in range(1,11): 
    route = recuperation_routes(f'/Users/augustincablant/Documents/GitHub/projetpython2023/ensae-prog23-main/input/routes.{i}.in')
    network =  graph_from_file(f'/Users/augustincablant/Documents/GitHub/projetpython2023/ensae-prog23-main/input/network.{i}.in')
    # Liste de chemin min_power
    liste_power = []
    for trajet in route: 
        liste_power.append(network.min_power_couvrant(trajet[0:2])[0])  # liste : [power,chemin]
    f = open(f'/Users/augustincablant/Documents/GitHub/projetpython2023/ensae-prog23-main/input/route.{i}.out','w',encoding='utf-8')
    for power in liste_power: 
        f.write(str(power)+'\n')
    f.close()

### Union Find

In [ ]:
################  Création de la classe Union Find  ################                 

class UF:
    def __init__(self,nodes):
        self.father = dict([(n, None) for n in nodes])
        self.rank = dict([(n, 0) for n in nodes])

    
    def find(self,node):
        if self.father[node]==None:
            return node
        return self.find(self.father[node]) 

    def union(self,nodex,nodey):
       xroot=self.find(nodex)
       yroot=self.find(nodey)
       if xroot != yroot:
            if self.rank[nodex] < self.rank[nodey]: 
                 self.father[nodex] = yroot
                 self.rank[nodex] += 1
            else:
                self.father[nodey] = xroot
                self.rank[nodey] += 1

### Fichier Graph 

In [ ]:
################  Fichier graph.py  ################   
############################# IMPORTATIONS ################################# 

import numpy as np
import matplotlib.pyplot as plt
from jyquickhelper import RenderJsDot
import pydot
from union_find import *
from time import perf_counter
from math import log2




############################# CLASSE Graph ################################# 
class Graph:
    def __init__(self, nodes=[]):
        self.nodes = nodes
        self.graph = dict([(n, []) for n in nodes])
        self.nb_nodes = len(nodes)
        self.nb_edges = 0
        self.edges = []
    
    #########

    def __str__(self):
        """Prints the graph as a list of neighbors for each node (one per line)"""
        if not self.graph:
            output = "The graph is empty"            
        else:
            output = f"The graph has {self.nb_nodes} nodes and {self.nb_edges} edges.\n"
            for source, destination in self.graph.items():
                output += f"{source}-->{destination}\n"
        return output
    
    #########

    #Méthode de la question 1 qui va nous permettre de compléter un graphe 
    def add_edge(self, node1, node2, power_min, dist=1):
        """
        Adds an edge to the graph. Graphs are not oriented, hence an edge is added to the adjacency list of both end nodes. 

        Parameters: 
        -----------
        node1: NodeType
            First end (node) of the edge
        node2: NodeType
            Second end (node) of the edge
        power_min: numeric (int or float)
            Minimum power on this edge
        dist: numeric (int or float), optional
            Distance between node1 and node2 on the edge. Default is 1.
        """
        self.nb_edges += 1
        self.graph[node1].append([node2, power_min, dist])
        self.graph[node2].append([node1, power_min, dist])
        return None
    
    #########

    # Création de la méthode get_path_with_power (question 3)
    # prend en entrée une puissance de camion p et un trajet t 
    # décide si un camion de puissance p peut couvrir le trajet t 
    # renvoie le chemin si possible

    def get_path_with_power(self, src, dest, power): # src, dest, power)
        # Vérification que les deux points sont dans la même composante fortement connexe
        composantes_connexes = self.connected_components()
        connectes=False
        for element in composantes_connexes:
            if (src and dest) in element: 
                composante = element
                connectes=True
        if not connectes :
            print("Les deux points ne sont pas connectés")
            return None
        #fin de la vérification
        
        # Trouver le chemin demandant le moins de "power" entre les deux points
        # Implémentation de l'algorithme de Djikstra
        
        
        cout = [float('inf')]*(len(self.nodes) +1)
        cout[src] = 0
        recuperer_chemin = [0]*(len(self.nodes) +1)
        
        #calcul du coût entre deux voisins dans une liste d'adjacence
        def calcul_cout(v1,v2):
            voisins2 = self.graph[v1]
            for element in voisins2:
                if element[0]==v2:
                    return element[1]
            raise ValueError("Les noeuds ne sont pas voisins")
             
        # implémentation d'un parcours en profondeur classique
        def explore_min_power(liste_comp):
            l=liste_comp
            
            while len(l) > 0:
                mini = cout[l[0]]
                for element in l:
                    
                    if cout[element]<=mini:
                        mini = cout[element]
                        sommet = element
                
                voisins = np.array(self.graph[sommet]).T[0]
                
                for voisin in voisins:
                    
                    cout_voisin = cout[sommet] + calcul_cout(sommet,voisin)
                    if cout_voisin < cout[voisin]:
                        cout[voisin] = cout_voisin
                        recuperer_chemin[voisin]=sommet
                    
                l.remove(sommet)
            chemin = [dest]
            while chemin[-1]!=src:
                chemin.append(recuperer_chemin[chemin[-1]])  
            return chemin[::-1]
        #cout est défini en variable donc on a accès au cout en faisant cout[dest]
        
        explore = explore_min_power(composante)
        
        if cout[dest] > power :
            print("Le camion ne dispose pas d'assez de puissance.")
            return None
        else:
            return explore
    
    #########
            
    def get_path_with_distance(self, src, dest): 

        # Vérification que les deux points sont dans la même composante fortement connexe
        composantes_connexes = self.connected_components()
        connectes=False
        for element in composantes_connexes:
            if (src and dest) in element: 
                composante = element
                connectes=True
        if not connectes :
            print("Les deux points ne sont pas connectés")
            return None
        #fin de la vérification
        
        # Trouver le chemin demandant le moins de distance entre les deux points
        # Implémentation de l'algorithme de Djikstra
            
            
        cout = [float('inf')]*(len(self.nodes) +1)
        cout[src] = 0
        
            
        recuperer_chemin = [0]*(len(self.nodes) +1)
            
        #calcul de la distance entre deux voisins dans une liste d'adjacence
        def calcul_distance(v1,v2):
            voisins2 = self.graph[v1]
            for element in voisins2:
                if element[0]==v2:
                    return element[2]
            raise ValueError("V1 et V2 ne sont pas voisins")         
            
        def explore_min_distance(liste_comp):
            l=liste_comp
                
            while len(l) > 0:
                mini = cout[l[0]]
                for element in l:
                        
                    if cout[element]<=mini:
                        mini = cout[element]
                        sommet = element
                    
                voisins = np.array(self.graph[sommet]).T[0]
                    
                for voisin in voisins:
                        
                    cout_voisin = cout[sommet] + calcul_distance(sommet,voisin)
                    if cout_voisin < cout[voisin]:
                        cout[voisin] = cout_voisin
                        recuperer_chemin[voisin]=sommet
                        
                l.remove(sommet)
            chemin = [dest]
            while chemin[-1]!=src:
                chemin.append(recuperer_chemin[chemin[-1]])  
            return chemin[::-1]
        #cout est défini en variable donc on a accès au cout en faisant cout[dest]
            
        explore = explore_min_distance(composante)
        return explore
    
    #  méthode connected_components_set qui trouve les composantes connectées du graphe
    # on va implémenter un parcours en profondeur du graphe en utilisant une fonction récursive

    def connected_components(self):
        composantes_connecte = []
        visite = {noeud : False for noeud in self.nodes}
        
        def dfs(noeud):
            composante = [noeud]
            for voisin in self.graph[noeud]:
                voisin = voisin[0]
                if not visite[voisin]:
                    visite[voisin] = True
                    composante += dfs(voisin)
            return composante
        
        for noeud in self.nodes:
            if not visite[noeud]:
                composantes_connecte.append(dfs(noeud))
        return composantes_connecte

    #########
    # calculer, pour un trajet t donné, la puissance minimale un camion pouvant couvrir ce trajet. 
    # La fonction retourne le chemin, et la puissance minimale.

    def min_power(self,trajet):
        #Vérifier qu'un camion peut bien effectuer le trajet
        # Pour cela on vérifie que tous les noeuds de trajet sont dans la même composante fortement connexe 

        composantes_connexes = self.connected_components()
        for noeud in trajet:
            for composante in composantes_connexes:
                if noeud not in composante:
                    composantes_connexes.remove(composante) 
        if len(composantes_connexes)<1:
            print("Le trajet proposé présente des noeuds qui ne sont pas connectés")
            return None
        else:

            #Il reste alors dans composantes_connexes seulement la composante_connexe qui nous intéresse 
            #Nous voulons désormais la puissance minimale de notre camion pour effectuer le trajet
            #On prend le chemin demandant le moins de "power"
            chemin = self.get_path_with_power(trajet[0], trajet[1], float('inf'))
            
            #On calcul la puissance de ce chemin
            def calcul_cout(v1,v2):
                voisins2 = self.graph[v1]
                for element in voisins2:
                    if element[0]==v2:
                        return element[1]
                raise ValueError("V1 et V2 ne sont pas voisins")
            
            power = 0
            for i in range(len(chemin)-1):
                power = max(power,calcul_cout(chemin[i],chemin[i+1]))
        return (power, chemin)
    

# Implémenter une représentation graphique du graphe, du trajet, et du chemin trouvé
# Optionnel

    #Pour l'instant cette méthode fonctionne uniquement sur jupyter 
    def plot_network(self):
        #on cherche à représenter le graphe
        
        rows = ["digraph{ ", '  rankdir="LR";']
        for i in range(1,self.nb_nodes+1):
            rows.append("  %d;" % i)     
        visited = [False]*(self.nb_nodes +1)
        visited[0]= True #on n'utilise pas cet emplacement
        for i in range(1,self.nb_nodes+1):
            visited[i]= True
            for j in range(len(self.graph[i])):
                if not visited[self.graph[i][j][0]]:
                    visited[self.graph[i][j][0]] = True
                    rows.append("  %d -> %d;" % (i, self.graph[i][j][0]))   
        rows.append("}")
        dot = "\n".join(rows)
        #print(dot)  # décommenter cette ligne pour voir le résultat
        return RenderJsDot(dot)
    
    #########

    def plot_network2(self):
        #on cherche à représenter le graphe

        rows = pydot.Dot(graph_type='digraph')

        for i in range(1,self.nb_nodes+1):
            rows.add_edge(pydot.Edge("  %d;" % i))   
        visited = [False]*(self.nb_nodes +1)
        visited[0]= True #on n'utilise pas cet emplacement
        for i in range(1,self.nb_nodes+1):
            visited[i]= True
            for j in range(len(self.graph[i])):
                if not visited[self.graph[i][j][0]]:
                    visited[self.graph[i][j][0]] = True
                    rows.add_edge(pydot.Edge("  %d -> %d;" % (i, self.graph[i][j][0]))) 
        rows.write_png('test.png')
        
    #########

    def plot_trajet(self,trajet):
        rows = ["digraph{ ", '  rankdir="LR";']
        for i in trajet:
            rows.append("%d;" %i)
        
        for i in range(len(trajet)-1):
            rows.append(" %d -> %d;" %(trajet[i],trajet[i+1]))
        
        rows.append("}")
        dot = "\n".join(rows)
        return RenderJsDot(dot)


    #  fonction kruskal : Graph -> Graph 
    #  Renvoie un arbre couvrant de poids minimal du graphe d'entrée. Analyser la complexité de l’algorithme implémenté.


    def kruskal(self):
        edges_sorted = sorted(self.edges, key = lambda arrete : arrete[2])
        arbre_couvrant = Graph(self.nodes)
        uf= UF(self.nodes) 
        for edge in edges_sorted:
            if uf.find(edge[0])!=uf.find(edge[1]):
                uf.find(edge[1])
                arbre_couvrant.add_edge(edge[0],edge[1],edge[2],edge[3])
                uf.union(edge[0],edge[1])
        return arbre_couvrant
    



    #Complexité de la fonction précédente - krustal :
    #   Notations : N = nb_nodes ; E = nb_edges
    #   Complexité du tri de python de la liste des arêtes : O(E log E) 
    #   Complexité de la boucle for : O( N*E ) avec un tour de boucle par arête pour une opération de find de coût N
    #   Bilan : Complexité Kruskal : O(E log E) + O( N*E ) = O( N*E )
    

    #########
   
    # calcule, pour un trajet t donné, la puissance minimale d’un camion pouvant couvrir ce trajet et le chemin associé
    def get_path_with_power_couvrant(self, src, dest, power): # src, dest, power)
        # Trouver le chemin demandant le moins de "power" entre les deux points
        # Implémentation de l'algorithme de Djikstra
        
        cout = [float('inf')]*(len(self.nodes) +1)
        cout[src] = 0
        recuperer_chemin = [0]*(len(self.nodes) +1)
        
        #calcul du coût entre deux voisins dans une liste d'adjacence
        def calcul_cout(v1,v2):
            voisins2 = self.graph[v1]
            for element in voisins2:
                if element[0]==v2:
                    return element[1]
            raise ValueError("Les noeuds ne sont pas voisins")
             
        
        def explore_min_power(liste_comp):
            l=liste_comp
            while len(l) > 0:
                mini = cout[l[0]]
                for element in l:
                    
                    if cout[element]<=mini:
                        mini = cout[element]
                        sommet = element
                
                voisins = np.array(self.graph[sommet]).T[0]
                
                for voisin in voisins:
                    
                    cout_voisin = cout[sommet] + calcul_cout(sommet,voisin)
                    if cout_voisin < cout[voisin]:
                        cout[voisin] = cout_voisin
                        recuperer_chemin[voisin]=sommet
                    
                l.remove(sommet)
            chemin = [dest]
            while chemin[-1]!=src:
                chemin.append(recuperer_chemin[chemin[-1]])  
            return chemin[::-1]
        #cout est défini en variable donc on a accès au cout en faisant cout[dest]
        explore = explore_min_power(self.nodes)
        
        if cout[dest] > power :
            print("Le camion ne dispose pas d'assez de puissance.")
            return None
        else:
            return explore

    def min_power_couvrant1(self,trajet):
        #trajet est une liste de deux éléments tq trajet[0] = point de départ 
        #trajet[1] = point d'arrivée 

        #on commence par chercher l'arbre couvrant 
        graph_arbre = self.kruskal()
        arbre_couvrant = graph_arbre.graph

        #on vérifie que le trajet existe : 
        composantes_connexes = graph_arbre.connected_components()
        for element in trajet:
            for composante in composantes_connexes:
                if element not in composante:
                    composantes_connexes.remove(composante) 
        if len(composantes_connexes)<1:
            print("Le trajet proposé présente des noeuds qui ne sont pas connectés")
            return None
        else: 
            #calcul du pcc avec Djikstra déjà implémenté
            chemin = graph_arbre.get_path_with_power_couvrant(trajet[0], trajet[1], float('inf'))

            #On calcul la "power" de ce chemin
            def calcul_cout(v1,v2):
                voisins2 = self.graph[v1]
                for element in voisins2:
                    if element[0]==v2:
                        return element[1]
                raise ValueError("V1 et V2 ne sont pas voisins")
            
            power = 0
            for i in range(len(chemin)-1):
                power = max(power,calcul_cout(chemin[i],chemin[i+1]))
        return (power,chemin)
    
    #Complexité de la fonction précédente - min_power_couvrant :
    #   Notations : N = nb_nodes ; E = nb_edges
    #   Complexité du calcul de connexité : O(E+N) [Algo DFS]
    #   Complexité de Kruskal : O(N*E) 
    #   Complexité de Djikstra : O( (E+N)*log N)=O( N * log N)  -> le nb de noeuds dans le graphe est N-1
    #   Bilan : Complexité de min_power_couvrant : O(N*E) + O(E+N) + O( N*log N) = O(E*N + N*log N) 
    #   NB : si l'on doit calculer tous les chemins du graphe (N!), le coût du calcul (unique) de Kruskal sera
    #           écrasé par le coût du calcul des chemins avec Djikstra -> au final, coût en O( N! * N *log N)...

    #   La complexité reste trop importante : l'algorithme ne tourne pas en pratique au delà de network1

        #########


    








    def profondeur(self,node, prof, parent, profondeurs, parents):
        '''Cette fonction est appelé dans la fonction parents_profondeurs. C'est elle qui fait les calculs.
        et modifie les deux dictionnaires pour renvoyer les dictionnaires parents et profondeurs.
        args :
            node(int): noeud dont on part pour s'enfoncer dans l'arbre
            prof(int): profondeur de node
            parent(int): parent de node
            g(objet de la classe graphe): graphe dont est issu node
            profondeurs(dictionbnaire): dictionnaire des profondeurs à modifier
            parents(dictionnaire): dictionnaires de parents à modifier 
        returns :
            profondeurs(dictionbnaire): dictionnaire des profondeurs ayant pour clefs les noeuds de l'arbre et pour valeurs les profondeurs associées
            parents(dictionnaire): dictionnaires de parents à modifier ayant pour clefs les noeuds de l'arbre et pour valeurs leur parent
    '''
        profondeurs[node]= prof #profondeurs est un dictionnaire, on ajoute donc la profondeur correspondante au noeud
        parents[node]= parent #parents est un dictionnaire, on ajoute le parent du noeud 

        for voisin in self.graph[node]  :
            if voisin[0] != parent : #on cherche à s'enfoncer dans l'arbre donc on ne veut pas retourner vers le parent
                self.profondeur(voisin[0], prof + 1, node, profondeurs, parents)#on réitère donc la fonction en prenant pour
                    #nouveau parent, le noeud voisin, on s'enfonce dans l'arbre, donc on augmente la profondeur de 1

#on code une fonction qui renvoie 2 dictionnaires dont les clefs sont les noeuds de l'arbre :
#un dictionnaire des profondeurs et un dictionnaires des parents
    def parents_profondeurs(self,racine) :
        '''cette fonction renvoie 2 dictionnaires dont les clefs sont les noeuds de l'arbre :
            un dictionnaire des profondeurs et un dictionnaires des parents.
            Args :
        racine(int): racine de l'arbre que l'on choisit
        g(objet de la classe graphe): arbre dont on veut connaitre les profondeurs et les parents de ses noeuds.
        '''
        profondeurs={} #on initialise
        parents ={} # on initialise

        self.profondeur(racine, 0, -1, profondeurs, parents) #par défault la racine à une profondeur de 0 et son parent est -1
        return profondeurs, parents





    def min_power_acm(self, src, dest):
        '''Le principe de cette fonction est de trouver le plus proche parent commun de dest et de src.
        On crée deux chemins, le premier relier la src à ce parent, le deuxième la destination au parent commun. 
        On concatène ensuite les deux chemins. On sait que ce chemin est unique par priopriété de kruskal'''
        profondeurs, parents = self.parents_profondeurs(1)


        #initialisation des chemins
        chemin1=[src] 
        chemin2=[dest]
        chemin=[]
        #pour trouver le parent commun, on a besoin de calculer les profondeurs de src et dest

        prof_dest= profondeurs[dest]
        prof_src= profondeurs[src]

        #Cas de bord où l'on veut relier src à elle-même
        if src == dest :
            return ([src],0)
        
        #le but est d'amener le noeud dont la profondeur est la plus grande
        #au niveau de la profondeur de l'autre noeud, pour remonter ensuite en même temps jusqu'au parent commun
        
        #disjonction de cas


        if prof_src>prof_dest :
            while prof_src!= prof_dest :
                src=parents[src]
                if src==dest :
                    chemin1.append(src)
                    chemin2=[]
                    break
                else :
                    prof_src = profondeurs[src]
                    chemin1.append(src)


        if prof_dest>prof_src :
            while prof_src!= prof_dest :
                dest=parents[dest]
                if src==dest:
                    chemin2.append(dest)
                    chemin1=[]
                    break
                
                else :
                    prof_dest = profondeurs[dest]
                    
                    chemin2.append(dest)
        
        #Une fois que les noeuds sont à la même hauteur, il se peut que le parent commun soit déjà trouvé
        #Le chemin le plus économique est donc trouvé
        if parents[src]==parents[dest] and src!=dest :
            src1=parents[src]
            
            chemin1.append(src1)

        #Une fois que les noeuds sont à la même hauteur, on remonte jusqu'au parent commun
        while parents[src]!=parents[dest]:
            chemin1.append(parents[src])
            chemin2.append(parents[dest])
            src=parents[src]
            dest=parents[dest]
        

            chemin1.append(parents[src])

        chemin2.reverse()
        
        chemin = chemin + chemin1 + chemin2
        
        #On cherche la puissance à renvoyer
        puissances = []
        p_min = -1 #si on ne trouve pas de chemin, on renvoie une puissance négative

        for i in range (len(chemin)-1):
            key = chemin[i]
            

            for e in self.graph[key]:

                if e[0]==chemin[i+1]:
                    puissances.append(e[1])
        
        if len(puissances)!=0 :
            p_min=max(puissances)

        return (chemin, p_min)




















    
        
############################# FONCTIONS #################################      
        

#Fonction qui permet de charger un fichier
def graph_from_file(filename):
    """
    Reads a text file and returns the graph as an object of the Graph class.

    The file should have the following format: 
        The first line of the file is 'n m'
        The next m lines have 'node1 node2 power_min dist' or 'node1 node2 power_min' (if dist is missing, it will be set to 1 by default)
        The nodes (node1, node2) should be named 1..n
        All values are integers.

    Parameters: 
    -----------
    filename: str
        The name of the file

    Outputs: 
    -----------
    G: Graph
        An object of the class Graph with the graph from file_name.
    """
    #récupération des données du file
    with open(filename) as file:
        ligne1 = file.readline().split()
        n = int(ligne1[0])
        m = int(ligne1[1])
        nodes = [i for i in range(1,n+1)]
        G = Graph(nodes)
        for i in range(m):
            ligne_i = file.readline().split()
            node1 = int(ligne_i[0])
            node2 = int(ligne_i[1])
            power_min = int(ligne_i[2])
            if len(ligne_i)>3:
                dist = int(ligne_i[3])
                G.add_edge(node1, node2, power_min, dist)
                G.edges.append([node1,node2,power_min,dist])
            else: 
                G.add_edge(node1,node2,power_min,dist=1)
                G.edges.append([node1,node2,power_min,1])
    return G






def estimation_temps(filename_reseau, filename_route, m=3):
    #création du graphe
    G = graph_from_file(filename_reseau)
    
    with open(filename_route) as file:
        liste_trajet = file.readlines()
        n = len(liste_trajet) - 1
        liste_trajet.remove(liste_trajet[0])
        nombre_aleatoire = np.random.randint(0,n,m)
        trajet_aleatoire = [liste_trajet[i].split()[:2] for i in nombre_aleatoire]
        
        for i in range(len(trajet_aleatoire)):
            for j in [0,1]:
                trajet_aleatoire[i][j] = int(trajet_aleatoire[i][j])
        
        t_start = perf_counter()
        for i in range(m):
            G.min_power(trajet_aleatoire[i])
        
        t_stop = perf_counter()
    
    return (n/m)*(t_stop - t_start)



#on cherche ici à obtenir une estimation correcte du temps de calcul sur le graphe avec kruskal

def estimation_temps_kruskal(filename_reseau, filename_route, m=3):

    #création du graphe
    G = graph_from_file(filename_reseau)
    
    with open(filename_route) as file:
        liste_trajet = file.readlines()
        n = len(liste_trajet) - 1
        liste_trajet.remove(liste_trajet[0])
        nombre_aleatoire = np.random.randint(0,n,m)
        trajet_aleatoire = [liste_trajet[i].split()[:2] for i in nombre_aleatoire]
        
        for i in range(len(trajet_aleatoire)):
            for j in [0,1]:
                trajet_aleatoire[i][j] = int(trajet_aleatoire[i][j])
        
        
        t_start = perf_counter()
        K=G.kruskal()
        for i in range(m):
            G.min_power(trajet_aleatoire[i])
        t_stop = perf_counter()
    
    return (n/m)*(t_stop - t_start)

    #########

In [ ]:
############################# CLASSE Graph ################################# 
class Graph:
    def __init__(self, nodes=[]):
        self.nodes = nodes
        self.graph = dict([(n, []) for n in nodes])
        self.nb_nodes = len(nodes)
        self.nb_edges = 0
        self.edges = []
    
    #########

    def __str__(self):
        """Prints the graph as a list of neighbors for each node (one per line)"""
        if not self.graph:
            output = "The graph is empty"            
        else:
            output = f"The graph has {self.nb_nodes} nodes and {self.nb_edges} edges.\n"
            for source, destination in self.graph.items():
                output += f"{source}-->{destination}\n"
        return output
    
    #########

    # Méthode de la question 1 qui va nous permettre de compléter un graphe 
    def add_edge(self, node1, node2, power_min, dist=1):
        """
        Adds an edge to the graph. Graphs are not oriented, hence an edge is added to the adjacency list of both end nodes. 

        Parameters: 
        -----------
        node1: NodeType
            First end (node) of the edge
        node2: NodeType
            Second end (node) of the edge
        power_min: numeric (int or float)
            Minimum power on this edge
        dist: numeric (int or float), optional
            Distance between node1 and node2 on the edge. Default is 1.
        """
        self.nb_edges += 1
        self.graph[node1].append([node2, power_min, dist])
        self.graph[node2].append([node1, power_min, dist])
        return None
    
    #########

    # Création de la méthode get_path_with_power (question 3)
    # prend en entrée une puissance de camion p et un trajet t 
    # décide si un camion de puissance p peut couvrir le trajet t 
    # renvoie le chemin si possible

    def get_path_with_power(self, src, dest, power): # src, dest, power)
        # Vérification que les deux points sont dans la même composante fortement connexe
        composantes_connexes = self.connected_components()
        connectes=False
        for element in composantes_connexes:
            if (src and dest) in element: 
                composante = element
                connectes=True
        if not connectes :
            print("Les deux points ne sont pas connectés")
            return None
        #fin de la vérification
        
        # Trouver le chemin demandant le moins de "power" entre les deux points
        # Implémentation de l'algorithme de Djikstra
        
        
        cout = [float('inf')]*(len(self.nodes) +1)
        cout[src] = 0
        recuperer_chemin = [0]*(len(self.nodes) +1)
        
        #calcul du coût entre deux voisins dans une liste d'adjacence
        def calcul_cout(v1,v2):
            voisins2 = self.graph[v1]
            for element in voisins2:
                if element[0]==v2:
                    return element[1]
            raise ValueError("Les noeuds ne sont pas voisins")
             
        # implémentation d'un parcours en profondeur classique
        def explore_min_power(liste_comp):
            l=liste_comp
            
            while len(l) > 0:
                mini = cout[l[0]]
                for element in l:
                    
                    if cout[element]<=mini:
                        mini = cout[element]
                        sommet = element
                
                voisins = np.array(self.graph[sommet]).T[0]
                
                for voisin in voisins:
                    
                    cout_voisin = cout[sommet] + calcul_cout(sommet,voisin)
                    if cout_voisin < cout[voisin]:
                        cout[voisin] = cout_voisin
                        recuperer_chemin[voisin]=sommet
                    
                l.remove(sommet)
            chemin = [dest]
            while chemin[-1]!=src:
                chemin.append(recuperer_chemin[chemin[-1]])  
            return chemin[::-1]
        #cout est défini en variable donc on a accès au cout en faisant cout[dest]
        
        explore = explore_min_power(composante)
        
        if cout[dest] > power :
            print("Le camion ne dispose pas d'assez de puissance.")
            return None
        else:
            return explore
    
    #########

    # Nous utilisons la même méthode pour un get_path_with_distance
    # Mêmes arguments 

    def get_path_with_distance(self, src, dest): 

        # Vérification que les deux points sont dans la même composante fortement connexe
        composantes_connexes = self.connected_components()
        connectes=False
        for element in composantes_connexes:
            if (src and dest) in element: 
                composante = element
                connectes=True
        if not connectes :
            print("Les deux points ne sont pas connectés")
            return None
        #fin de la vérification
        
        # Trouver le chemin demandant le moins de distance entre les deux points
        # Implémentation de l'algorithme de Djikstra
            
            
        cout = [float('inf')]*(len(self.nodes) +1)
        cout[src] = 0
        
            
        recuperer_chemin = [0]*(len(self.nodes) +1)
            
        #calcul de la distance entre deux voisins dans une liste d'adjacence
        def calcul_distance(v1,v2):
            voisins2 = self.graph[v1]
            for element in voisins2:
                if element[0]==v2:
                    return element[2]
            raise ValueError("V1 et V2 ne sont pas voisins")         
            
        def explore_min_distance(liste_comp):
            l=liste_comp
                
            while len(l) > 0:
                mini = cout[l[0]]
                for element in l:
                        
                    if cout[element]<=mini:
                        mini = cout[element]
                        sommet = element
                    
                voisins = np.array(self.graph[sommet]).T[0]
                    
                for voisin in voisins:
                        
                    cout_voisin = cout[sommet] + calcul_distance(sommet,voisin)
                    if cout_voisin < cout[voisin]:
                        cout[voisin] = cout_voisin
                        recuperer_chemin[voisin]=sommet
                        
                l.remove(sommet)
            chemin = [dest]
            while chemin[-1]!=src:
                chemin.append(recuperer_chemin[chemin[-1]])  
            return chemin[::-1]
        #cout est défini en variable donc on a accès au cout en faisant cout[dest]
            
        explore = explore_min_distance(composante)
        return explore
    
    #########


    #  méthode connected_components_set qui trouve les composantes connectées du graphe
    # on va implémenter un parcours en profondeur du graphe assez classique en utilisant une fonction récursive

    def connected_components(self):
        composantes_connecte = []
        visite = {noeud : False for noeud in self.nodes}
        
        def dfs(noeud):
            composante = [noeud]
            for voisin in self.graph[noeud]:
                voisin = voisin[0]
                if not visite[voisin]:
                    visite[voisin] = True
                    composante += dfs(voisin)
            return composante
        
        for noeud in self.nodes:
            if not visite[noeud]:
                composantes_connecte.append(dfs(noeud))
        return composantes_connecte
    

    #########
    # calculer, pour un trajet t donné, la puissance minimale un camion pouvant couvrir ce trajet. 
    # La fonction retourne le chemin, et la puissance minimale.

    def min_power(self,trajet):
        #Vérifier qu'un camion peut bien effectuer le trajet
        # Pour cela on vérifie que tous les noeuds de trajet sont dans la même composante fortement connexe 

        composantes_connexes = self.connected_components()
        for noeud in trajet:
            for composante in composantes_connexes:
                if noeud not in composante:
                    composantes_connexes.remove(composante) 
        if len(composantes_connexes)<1:
            print("Le trajet proposé présente des noeuds qui ne sont pas connectés")
            return None
        else:

            #Il reste alors dans composantes_connexes seulement la composante_connexe qui nous intéresse 
            #Nous voulons désormais la puissance minimale de notre camion pour effectuer le trajet
            #On prend le chemin demandant le moins de "power"
            chemin = self.get_path_with_power(trajet[0], trajet[1], float('inf'))
            
            #On calcul la puissance de ce chemin
            def calcul_cout(v1,v2):
                voisins2 = self.graph[v1]
                for element in voisins2:
                    if element[0]==v2:
                        return element[1]
                raise ValueError("V1 et V2 ne sont pas voisins")
            
            power = 0
            for i in range(len(chemin)-1):
                power = max(power,calcul_cout(chemin[i],chemin[i+1]))
        return (power, chemin)
    

    #########

    #Implémenter une représentation graphique du graphe, du trajet, et du chemin trouvé. 
"""""
    def to_graphviz(self, comment="Graphe", view=True, src=None, dest=None):  
        path = []
        if src is not None and dest is not None:
            path, _ = self.min_power(src, dest)
            if path is None:
                path = []
        path_couple = [{path[i], path[i+1]} for i in range(len(path)-1)]
        
        dot = graphviz.Graph(comment=comment)
        for node in self.nodes:
            if node == src:
                dot.node(f"{node}", str(node), color="forestgreen")
            elif node == dest:
                dot.node(f"{node}", str(node), color="firebrick")
            elif node in path:
                dot.node(f"{node}", str(node), color="dodgerblue")
            else:
                dot.node(f"{node}", str(node))
        for source, destinations in self.graph.items():
            for destination, power, dist in destinations:
                if source < destination:
                    # Le premier nombre est la puissance le deuxième est la distance
                    if {source, destination} in path_couple:
                        dot.edge(str(source), str(destination), label=f'{power}, {dist}', color="dodgerblue")
                    else:
                        dot.edge(str(source), str(destination), label=f'{power}, {dist}')
        dot.render(filename=f'graphviz/{comment}.gv', cleanup=True, view=view)

"""
############################# FONCTIONS #################################      
        

#Fonction qui permet de charger un fichier (c'est la question 4)
def graph_from_file(filename):
    #récupération des données du file
    with open(filename) as file:
        ligne1 = file.readline().split()
        n = int(ligne1[0])
        m = int(ligne1[1])
        nodes = [i for i in range(1,n+1)]
        G = Graph(nodes)
        for i in range(m):
            ligne_i = file.readline().split()
            node1 = int(ligne_i[0])
            node2 = int(ligne_i[1])
            power_min = int(ligne_i[2])
            if len(ligne_i)>3:
                dist = int(ligne_i[3])
                G.add_edge(node1, node2, power_min, dist)
                G.edges.append([node1,node2,power_min,dist])
            else: 
                G.add_edge(node1,node2,power_min,dist=1)
                G.edges.append([node1,node2,power_min,1])
    return G


############################# Kruskal et ses amis ################################# 

def find(parent, x):
    # trouver le parent de x, (représentant de l'ensemble auquel il appartient)
    """""
    parent(list): liste répertoriant les parents de chaque noeud. le parent de i est parent[i]
    x(int): noeud dont on veut trouver le parent
    returns: parent[x](int): parent de x
    """""
    if parent[x] != x:
        parent[x]=find(parent, parent[x])
    return parent[x]
    
################# 

def kruskal(g):
    # arbre couvrant minimal 
    g_mst=Graph() 
    mst_edges=[]
    aretes=[]
    
    for node in g.nodes: #Parcours toutes les arrêtes du graphe
        for e in g.graph[node]:
            ar = (node,) + e 
            aretes.append(ar)
            
    aretes.sort(key=lambda x : x[2])

    parent = list(range(g.nb_nodes)) #on initialise la liste représentant les sous-ensembles, chaque noeud est parent de lui-même à l'initialisation

    compt=0 #un arbre a au max nb_nodes -1 arêtes, donc on ajoute une variable compteur

    for ar in aretes :
        n1= ar[0] -1 
        n2=ar[1] -1
        a=ar[0]

        parent_n1=find(parent, n1)
        parent_n2=find(parent, n2)

        if parent_n1 != parent_n2 :
            parent[parent_n1] = parent_n2 
            mst_edges.append(ar)
      
    g_mst = Graph(g.nodes)
    

    for i in range (0,len(mst_edges)):
        ar=mst_edges[i]
        a=ar[0]
        b=ar[1]
        p=ar[2] 
        if len(ar)>3:
            dist=ar[3]
            g_mst.add_edge(a, b , p, dist)
        else :
            g_mst.add_edge(a, b , p)
    
    return g_mst

################# 

# Trouver le plus court chemin dans un arbre minimal couvrant 

#Calcul des parents et des profondeurs des noeuds de l'arbre

def profondeur(node, prof, parent, g, profondeurs, parents):
    profondeurs[node]= prof 
    parents[node]= parent 

    for voisin in g.graph[node] :
        if voisin[0] != parent : 
            profondeur(voisin[0], prof +1, node, g, profondeurs, parents)


def parents_profondeurs(racine, g) :
    profondeurs={} #on initialise
    parents ={} # on initialise

    profondeur(racine, 0, -1, g, profondeurs, parents) 
    return profondeurs, parents # un dictionnaire des profondeurs et un dictionnaires des parents



def min_power_update(g_mst, src, dest):
    # On utilise l'arbre couvrant pour calculer la puissance minimale
    chemin=[src]
    profondeurs, parents = parents_profondeurs(dest,g_mst) 
    p_src = profondeurs[src]
    for i in range (p_src):
        chemin.append(parents[src])
        src= parents[src]
    return chemin


def min_power_update2(src, dest, g_mst, parents, profondeurs):
    # T rouver le plus proche parent commun de dest et de src.
    # On crée deux chemins, le premier relier la src à ce parent, le deuxième la destination au parent commun. 
    # On concatène ensuite les deux chemins. On sait que ce chemin est unique et le 
    # plus économique en puissance par priopriété de kruskal

    #initialisation des chemins
    chemin1=[src] 
    chemin2=[dest]
    chemin=[]

    prof_dest= profondeurs[dest]
    prof_src= profondeurs[src]

    #Cas de bord où l'on veut relier src à elle-même
    if src == dest :
        return ([src],0)
    
    #le but est d'amener le noeud dont la profondeur est la plus grande
    #au niveau de la profondeur de l'autre noeud, pour remonter ensuite en même temps jusqu'au parent commun
    if prof_src>prof_dest :
        while prof_src!= prof_dest :
            src=parents[src]
            if src==dest :
                chemin1.append(src)
                chemin2=[]
                break
            else :
                prof_src = profondeurs[src]
                chemin1.append(src)


    if prof_dest>prof_src :
        while prof_src!= prof_dest :
            dest=parents[dest]
            if src==dest:
                chemin2.append(dest)
                chemin1=[]
                break
            
            else :
                prof_dest = profondeurs[dest]
                
                chemin2.append(dest)
    
    if parents[src]==parents[dest] and src!=dest :
        src1=parents[src]
        
        chemin1.append(src1)

    #Une fois que les noeuds sont à la même hauteur, on remonte jusqu'au parent commun
    while parents[src]!=parents[dest]:
        chemin1.append(parents[src])
        chemin2.append(parents[dest])
        src=parents[src]
        dest=parents[dest]
    

        chemin1.append(parents[src])

    chemin2.reverse()
    chemin = chemin + chemin1 + chemin2

    puissances = []
    p_min = -1 

    for i in range (len(chemin)-1):
        key = chemin[i]
        

        for e in g_mst.graph[key]:

            if e[0]==chemin[i+1]:
                puissances.append(e[1])
    
    if len(puissances)!=0 :
        p_min=max(puissances)
    return (chemin, p_min)
# chemin : liste contenant les noeuds composant le chemin le plus économique en puissance entre src et dest
# p_min : puissance minimale que le camion doit posséder pour rejoindre les deux noeuds.

### Fichier route 

In [ ]:
################  Fichier routeout.py  ################   
############################# IMPORTATIONS ################################# 
from graph2 import *


############################# Fonctions ################################# 
def routes_out(i):
     # générer le fichier route.out (T lignes : sur chaque ligne, puissance minimale pour parcourir la route et utilité)
    filename="ensae-prog23-main/input/routes."+ str(i) + ".in"
    filename_nv="routes."+ str(i) + ".out"
    network = "ensae-prog23-main/input/network." + str(i)+ ".in"
    
    g= graph_from_file(network)
    g_mst= kruskal(g)
    
    profondeurs, parents = parents_profondeurs(1, g_mst)
    res=[]

    with open(filename) as file: #on ouvre le fichier
            ligne1=file.readline().split()
            nb_trajets=int(ligne1[0])

            for i in range(nb_trajets):
                lignei=file.readline().split() # i eme ligne du fichier file 
                utilite=str(lignei[2]) #on récupère l'utilité
                src=int(lignei[0]) # Premier noeud à relier 
                dest=int(lignei[1]) # Second noeud à relier

                chemin, p_min = min_power_update2(src, dest, g_mst, parents, profondeurs)

                res.append((p_min, utilite))

    with open(filename_nv, "w") as file :
        for  el in res :
            puissance = el[0]
            utilite = el[1]
            file.write(str(puissance)+ " " + utilite + "\n")


for i in range(1,11):
    print(routes_out(i))

print(routes_out(2))

### Fichier Camion 

In [ ]:
from graph2 import *


############################# Fonctions ################################# 


#Récupération de la liste des camions :

def catalogue_file_in(file):
    # Renvoyer le catalogue de camion sous forme de liste à partir du fichier trucks.x.in.
    filename="input/trucks."+ str(file) + ".in"
    res=[]
    with open(filename) as file: 
        ligne1=file.readline().split()
        nb_trucks=int(ligne1[0])
        for i in range(nb_trucks):
            lignei=file.readline().split() 
            puissance=int(lignei[0]) 
            cout=int(lignei[1]) 
            res.append((puissance, cout))
    return(res)

####################

def catalogue_file_out(file):
    # Renvoyer le catalogue de camions utiles sous forme de liste à partir du fichier trucks_utile.x.out.
    filename="input/trucks_utile."+ str(file) + ".out"
    res=[]
    with open(filename) as file: #on ouvre le fichier
        ligne1=file.readline().split()
        nb_trucks=int(ligne1[0])

        for i in range(nb_trucks):
            lignei=file.readline().split() # i eme ligne du fichier file 
            puissance=int(lignei[0]) # Puissance du camion 
            cout=int(lignei[1]) # coût du camion

            res.append((puissance, cout))
    return(res)

####################


#Récupération des routes sous forme de liste

def routes_file_out(x):
    # Renvoyer le fichier route out sous forme de liste
    filename="input/routes."+ str(x) + ".out"
    filenamein="input/routes."+ str(x) + ".in"
    res=[]
    with open(filenamein) as file: #on ouvre le fichier
        ligne1=file.readline().split()
        nb_trajets=int(ligne1[0])
    with open(filename) as file : 
        for i in range(nb_trajets):
            lignei=file.readline().split() 
            puissance_min=int(lignei[0]) 
            utilite=float(lignei[1]) 
            res.append((puissance_min, utilite))
    return(res)

####################

def catalogue_puiss_raiso(x):
    # Renvoyer un catalogue avec des camions ayant une puissance "raisonnable"
    catalogue= catalogue_file_in(x)
    prix_tmp=catalogue[-1][1]
    catalogue_puiss_raiso=[catalogue[-1]]
    for i in range (len(catalogue)-1):
        j=len(catalogue)-i-1 
        if catalogue[j-1][1]<catalogue_puiss_raiso[0][1]: 
            catalogue_puiss_raiso.insert(0,catalogue[j-1])
            prix_tmp=catalogue[j-1][1]

    filename_new="input/trucks_utile."+ str(x) + ".out"
    with open(filename_new, "w") as file :
        a=str(len(catalogue_puiss_raiso))
        file.write(a+"\n")
        for  element in catalogue_puiss_raiso :
            puissance = element[0]
            cout = element[1]
            file.write(str(puissance)+ " " + str(cout) + "\n")
            
####################

def appariement(x_trucks,y_routes):
    # camion le moins cher pour chaque trajet du fichier routes.out
    catalogue = catalogue_file_out(x_trucks)
    routes = routes_file_out(y_routes)
    appariement=[]
    for i in range (len(routes)):
        p_min=routes[i][0]
        a=0
        p1=catalogue[a][0]
        if p1 >= p_min:
            element=(p1, catalogue[a][1], routes[i][1])
            appariement.append(element)
        b=len(catalogue)-1
        p2=catalogue[b][0]
        m=(a+b)//2
        pm=catalogue[m][0]
    
        while a<=b :               
            if pm==p_min:
                el=(pm, catalogue[m][1], routes[i][1])
                appariement.append(element)
                break
            elif pm<p_min :
                a=m+1
            elif pm>p_min and catalogue[m-1][0]<p_min :
                element =(pm, catalogue[m][1], routes[i][1])
                appariement.append(element)
                break
            elif pm>p_min:
                b=m-1
            m=(a+b)//2
            pm=catalogue[m][0]  
    return(appariement) 
# format : (puissance camion sélectionné, coût du camion selectionné, utilité de la route à traverser avec le camion sélectionné)


####################

def temps_appariement(x_trucks,y_routes):
    debut = perf_counter()
    appariement(x_trucks,y_routes)
    fin = perf_counter()
    tmp = fin- debut
    return("Le temps de appariement est", tmp, "s")


####################

def bagpack(budget, appariement):
    appariement_tri = sorted(appariement, key=lambda x: x[2]) 
    selection = []
    cout_total = 0
    while appariement_tri:
        element = appariement_tri.pop() 
        if element[1] + cout_total <= budget: 
            selection.append(element)
            cout_total += element[1]
    gain = sum([i[2] for i in selection])
    return gain, selection  
# (gain réalisé par l'entreprise, selection de camions à commander avec leur prix et l'affectation à une route désignée )
# format sélection : [puissance camion, coût camion, utilité route affectée au camion]



####################

def bagpack_update(budget, appariement):
    # Update : au lieu de trier l'appariement par utilité, on trie par utilié/côut
    appariement_tri = sorted(appariement, key=lambda x: x[2]/x[1]) 
    selection = []
    cout_total = 0
    while appariement_tri:
        element = appariement_tri.pop() 
        if element[1] + cout_total <= budget: 
            selection.append(element) 
            cout_total += element[1] 
    gain = sum([i[2] for i in selection])
    return gain, selection 


####################



# Recherche de toutes les solutions
def bagpack_force_brute(budget, appariement, selection = []):
    # Fonction très très coûteuse ....
    if appariement: 
        element1 = bagpack_force_brute(budget, appariement[1:], selection)[0]
        liste_element1 = bagpack_force_brute(budget, appariement[1:], selection)[1] 
        element = appariement[0] 
        if element[1] <= budget: #on regarde si on peut acheter le camion
            element2 = bagpack_force_brute(budget - element[1], appariement[1:], selection + [element])[0]
            liste_element2 = bagpack_force_brute(budget - element[1], appariement[1:], selection + [element])[0]
            #Si c'est le cas on l'ajoute à la selection, on retire le coût du camion au budget et on retire le camion de la liste appariement
            if element1 < element2: 
                return element2, liste_element2
        return element1, liste_element1
    else:
        profit= sum([i[2] for i in selection]) #on calcule le profit qui est la somme des utilités
        return profit, selection



####################

In [ ]:
################  CréaFichier trucks.py  ################   


from graph import *

##### FONCTIONS #####    


# QUESTION 18
def collection_camions(filename):
    with open(filename) as file:
        nb_modele = int(file.readline())
        liste_camions = []
        for i in range(int(nb_modele)):
            ligne = file.readline().split()
            liste_camions.append([int(ligne[0]),int(ligne[1])])
        return liste_camions
    

def recuperation_routes(filename):
    with open(filename) as file: 
        nb_routes = int(file.readline())
        liste_routes = []
        for i in range(int(nb_routes)):
            ligne = file.readline().split()
            liste_routes.append([int(ligne[0]),int(ligne[1]), int(ligne[2])])
        return liste_routes



## Fonctions nécessaires pour la suite : détermination du meilleur camion pour couvrir un chemin


#format de la liste camions en entrée : camions[i] = puissance, coût, nom ; on suppose ici que la liste des camions est triée par puissance en entrée -> complexité en log(len(camions))
#renvoie l'ensemble des camions capables de couvrir un chemin d'une puissance donnée

def camions_couvrant_dicho(puiss, camions):
    n = len(camions)
    x1=0
    x2=n-1
    if camions[n-1][0]-puiss<0:
        return [] #aucun camion ne convient
    if camions[0][0]-puiss>=0:
        return camions #tous les camions conviennent
    
    while True :
        x=(x1+x2)//2 
        if camions[x][0]-puiss==0:
            return camions[x:] #tous les camions de puissance supérieure conviennent
        elif camions[x][0]-puiss<0:
            x1 = x 
        else: 
            if camions[x-1][0]-puiss<0:
                return camions[x:] 
            else:
                x2=x

#format de la liste camions en entrée : camions[i] = puissance, coût, nom ; on suppose ici que la liste des camions est triée par puissance en entrée
#renvoie le camion de coût minimnal

def camion_couvrant_opti(puiss,camions):      
    cam=camions_couvrant_dicho(puiss, camions)
    if cam==[]:
        return -1  #aucun camion ne convient
    else : 
         return min(cam,key=lambda camion: camion[1])[2]


def format(file_network,file_routein,file_camion): 
    network = graph_from_file(file_network)
    
    #format chemin
    routes = recuperation_routes(file_routein)
    chemins = []
    for route in routes:
        chemin_min_power = network.min_power_acm(route[0],route[1])
        chemins.append([chemin_min_power[0],chemin_min_power[1],route[2]])

    #format camion
    camions = []
    collection = collection_camions(file_camion)
    i = 0
    for camion in collection:
        camions.append([camion[0],camion[1],i])
        i +=1
    return (chemins,camions)

### Algorithme glouton 


#format de la liste chemins en entrée : chemins[i] = [chemin = liste de noeuds], puissance du chemin, utilité du chemin
#format de la liste camions en entrée : camions[i] = puissance, coût, nom = position dans le catalogue (important pour ne pas les perdre en triant)

def knapsack_glouton(chemins,camions,budget=10**9):
    chemins_triee=sorted(chemins, key=lambda chemin: chemin[2])
    min_cout = min(camions,key=lambda camion: camion[1])[1]
    achats_camions=np.zeros(len(camions))
    chemins_couverts=[]
    utilite=0
    budget_restant=budget
    i=0
    while budget_restant>min_cout:
        puissancei=chemins_triee[i][1]
        camion=camion_couvrant_opti(puissancei,camions) 
        if camion==-1: #cas où aucun camion ne convient 
            continue
        elif camions[camion][1]<budget_restant:
            budget_restant-=camions[camion][1]
            achats_camions[camion]+=1
            chemins_couverts.append(chemins_triee[i])
            utilite+=chemins_triee[i][2]
    return(chemins_couverts,achats_camions,budget-budget_restant,utilite)





### Algorithme de force brute 
# Principe : L'idée est de parcourir l'ensemble des possibilités représentées par des clés binaires, faciles à gérer

#format de la liste chemins en entrée : chemins[i] = [chemin = liste de noeuds], utilité du chemin, coût de couverture optimale, nom du camion de couverture optimale


def utilite(combinaison):
    u=0
    for chemin in combinaison:
        u+=chemin[1]
    return u


def cout(combinaison):
    c=0
    for chemin in combinaison:
        c+=chemin[2]
    return c



#format de la liste chemins en entrée : chemins[i] = [chemin = liste de noeuds], puissance du chemin, utilité du chemin
#format de la liste camions en entrée : camions[i] = puissance, coût, nom = position dans le catalogue 
# (important pour ne pas les perdre en triant)

def pre_process(chemins,camions):
    chemins_processed=[]
    for chemin in chemins:
        chemin_processed=chemin
        camion_opti=camion_couvrant_opti(chemin[1],camions)
        if camion_opti != -1: #cas où aucun camion ne convient
            chemin_processed.append(camions[camion_opti][1])
            chemin_processed.append(camion_opti)
            chemins_processed.append(chemin_processed)
    return(chemins_processed)


#format de la liste chemins en entrée : chemins[i] = [chemin = liste de noeuds], utilité du chemin, coût de couverture optimale, nom du camion de couverture optimale

def post_process(chemins,camions):
    chemins_couverts=[]
    achats_camions=np.zeros(len(camions))
    for chemin in chemins:
        chemins_couverts.append(chemin[0])
        achats_camions[chemin[3]]+=1
    return chemins_couverts, achats_camions




#format de la liste chemins en entrée : chemins[i] = [chemin = liste de noeuds], puissance du chemin, utilité du chemin
#format de la liste camions en entrée : camions[i] = puissance, coût, nom = position dans le catalogue (important pour ne pas les perdre en triant)

def force_brute(chemins,camions,budget=10**9):
    n = len(chemins)
    univers = 2 ** n 
    meilleure_combinaison = []
    cout_mc=0
    utilite_mc=0


    # PRE PROCESSING pour le format
    chemins_processed = pre_process(chemins,camions)
 
    for cle in range(univers): 
        
        chaine_binaire = bin(cle)[2:]
        long = len(chaine_binaire)
        if long < n:
            chaine_binaire = (n - long) * '0' + chaine_binaire 

        combinaison = []
        for i in range(n):
            if chaine_binaire[i] == '1':
                combinaison.append(chemins_processed[i])
        
        utilite_temp = utilite(combinaison)
        cout_temp = cout(combinaison)

        if utilite_temp > utilite_mc:
            if cout_temp < budget:
                meilleure_combinaison = combinaison
                cout_mc=cout_temp
                utilite_mc=utilite_temp

        # POST PROCESSING pour le format
        chemins_couverts,achats_camions=post_process(meilleure_combinaison,camions)
        return(chemins_couverts,achats_camions,cout_mc,utilite_mc)
    

### Estimation du temps

In [ ]:
################  Fichier estimation_temps.py  ################   
############################# IMPORTATIONS ################################# 

from graph2 import *
from time import*

# Nous traitons la question 10 en estimant le temps nécessaire pour calculer la puissance minimale 
# sur l'ensemble des trajets pour chacun des fichiers routes.x.in



######On définit une fonction qu'on peut appliquer à chaque fichier routes######


def temps_meth1(file_route):

    filename="input/routes."+ str(file_route) + ".in"
    network = "input/network." + str(file_route)+ ".in"

    with open(filename) as file: #on ouvre le fichier
            ligne1=file.readline().split()
            nb_trajets=int(ligne1[0])
        
        
            ligne2=file.readline().split() # 2 eme ligne du fichier file 
            src2=int(ligne2[0]) # Premier noeud à relier 
            dest2=int(ligne2[1]) # Second noeud à relier

            ligne3=file.readline().split() # 3 eme ligne du fichier file 
            src3=int(ligne3[0]) # Premier noeud à relier 
            dest3=int(ligne3[1]) # Second noeud à relier

            ligne4=file.readline().split() # 4 eme ligne du fichier file 
            src4=int(ligne4[0]) # Premier noeud à relier 
            dest4=int(ligne4[1]) # Second noeud à relier



    g = graph_from_file(network)

    #on le teste sur les 3 premières lignes du fichier et on fait une moyenne de ces 3 temps pour ensuite
    #la multiplier par le nombre de trajet

    debut2 = perf_counter()
    g.min_power(src2,dest2)
    fin2 = perf_counter()
    tmp2= fin2 - debut2


    debut3 = perf_counter()
    g.min_power(src3,dest3)
    fin3 = perf_counter()
    tmp3= fin3 - debut3
    

    debut4 = perf_counter()
    g.min_power(src4,dest4)
    fin4 = perf_counter()
    tmp4= fin4 - debut4

    tmp_moy = (tmp2 + tmp3 +tmp4)/3
    temps_routes = tmp_moy*nb_trajets

    return ("Le temps pour calculer l'ensemble des trajets du fichier routes", str(file_route), "est", temps_routes, "s")


# Durée de plusieurs jours pour les fichiers 1 et 2 : beaucoup trop long !!!


###################### Deuxième méthode ######################


def temps_meth2(file_route):

    filename="input/routes."+ str(file_route) + ".in"
    network = "input/network." + str(file_route)+ ".in"

    with open(filename) as file: #on ouvre le fichier
            ligne1=file.readline().split()
            nb_trajets=int(ligne1[0])
        
        
            ligne2=file.readline().split() # 2 eme ligne du fichier file 
            src2=int(ligne2[0]) # Premier noeud à relier 
            dest2=int(ligne2[1]) # Second noeud à relier

            ligne3=file.readline().split() # 3 eme ligne du fichier file 
            src3=int(ligne3[0]) # Premier noeud à relier 
            dest3=int(ligne3[1]) # Second noeud à relier

            ligne4=file.readline().split() # 4 eme ligne du fichier file 
            src4=int(ligne4[0]) # Premier noeud à relier 
            dest4=int(ligne4[1]) # Second noeud à relier



    g = graph_from_file(network)
    g1=kruskal(g)

    debut2 = perf_counter()
    min_power_update(g1,src2,dest2)
    fin2 = perf_counter()
    tmp2= fin2 - debut2


    debut3 = perf_counter()
    min_power_update(g1,src3,dest3)
    fin3 = perf_counter()
    tmp3= fin3 - debut3
    

    debut4 = perf_counter()
    min_power_update(g1, src4,dest4)
    fin4 = perf_counter()
    tmp4= fin4 - debut4

    tmp_moy = (tmp2 + tmp3 +tmp4)/3
    temps_routes = tmp_moy*nb_trajets

    return ("Le temps pour calculer l'ensemble des trajets du fichier routes avec la deuxième fonction", str(file_route), "est",temps_routes, "s")

# Réduction du temps à 2h pour les premiers network 1 et 2.
# Le programme a du mal à calculer les résultats pour les plus gros graphes ...




###################### Troisième méthode ######################

# On définit une fonction qui calcule le temps pour trouver les trajets sur un fichier route avec la troisième méthode
# comme indiqué dans la séance 3 

def temps_meth3(file_route):

    filename="input/routes."+ str(file_route) + ".in"
    network = "input/network." + str(file_route)+ ".in"

    with open(filename) as file: #on ouvre le fichier
            ligne1=file.readline().split()
            nb_trajets=int(ligne1[0])
        
        
            ligne2=file.readline().split() # 2 eme ligne du fichier file 
            src2=int(ligne2[0]) # Premier noeud à relier 
            dest2=int(ligne2[1]) # Second noeud à relier
        

            ligne3=file.readline().split() # 3 eme ligne du fichier file 
            src3=int(ligne3[0]) # Premier noeud à relier 
            dest3=int(ligne3[1]) # Second noeud à relier

            ligne4=file.readline().split() # 4 eme ligne du fichier file 
            src4=int(ligne4[0]) # Premier noeud à relier 
            dest4=int(ligne4[1]) # Second noeud à relier



    g = graph_from_file(network)
    g1=kruskal(g)
    
    po, pa = parents_profondeurs(1, g1)

    debut2 = perf_counter()
    a=min_power_update2(src2,dest2, g1, pa, po)
    fin2 = perf_counter()
    tmp2= fin2 - debut2
    


    debut3 = perf_counter()
    b=min_power_update2(src3,dest3, g1, pa, po)
    fin3 = perf_counter()
    tmp3= fin3 - debut3
    
    

    debut4 = perf_counter()
    c=min_power_update2(src4,dest4, g1, pa, po)
    fin4 = perf_counter()
    tmp4= fin4 - debut4
    

    tmp_moy = (tmp2 + tmp3 +tmp4)/3
    temps_routes = tmp_moy*nb_trajets

    return (temps_routes)


# Temps Kruskal 

def temps_meth3_krusk(file_route):
    network = "input/network." + str(file_route)+ ".in"
    g=graph_from_file(network)

    debut = perf_counter()
    kruskal(g)
    fin = perf_counter()
    tmp = fin- debut

    return("Le temps de Kruskal est", tmp, "s")


# Temps parents_profondeurs / kruskal


def temps_meth3_kruskal_prof_par(file_route):
    network = "input/network." + str(file_route)+ ".in"
    g=graph_from_file(network)

    debut = perf_counter()
    g_mst=kruskal(g)
    parents_profondeurs(1,g_mst)
    fin = perf_counter()
    tmp = fin- debut

    return(tmp)


def temps_total(file_route) :
    tmpmin = temps_meth3(file_route)
    t_k_par_prof= temps_meth3_kruskal_prof_par(file_route)
    total = tmpmin + t_k_par_prof
    return ("Le temps pour calculer l'ensemble des trajets du fichier routes au total du fichier route", str(file_route), "est",total, "s")


"""""
def comparaison(file_route):
    print(temps_meth1(file_route))
    print(temps_meth3(file_route))
    print(calcul(file_route))
    print(temps_meth3_kruskal_prof_par(file_route))
    print(temps_total(file_route))
"""""

# Temps inférieur à 3 minutes 30 : c'est optimal !